# Introduction

Python is an interpreted, high-level, general-purpose programming language.

# Inheritance

Python support inheritance as any other object oriented language. Python also supports multiple inheritance. But with multiple, the diamond problem of method resolution occurs. For example consider the following example.

In [7]:
class GrandParent(object): pass

class ParentA(GrandParent): 
    
    def say_hello(self):
        print('Hello from Parent A')

class ParentB(GrandParent):
    
    def say_hello(self):
        print('Hello from Parent B')

class Child(ParentA, ParentB): pass


child1 = Child()
child1.say_hello()

# MRO of Child class
print()
print('MRO for child class: ')
print(Child.mro())

Hello from Parent A

MRO for child class: 
[<class '__main__.Child'>, <class '__main__.ParentA'>, <class '__main__.ParentB'>, <class '__main__.GrandParent'>, <class 'object'>]


Here, class `Child` inherits from two classes `ParentA` and `ParentB` and each of them have a method named `say_hello()`. So when we called `say_hello` on an instance of `Child` class which method should get resolved?

Python solves this ambiguity with something called as **Method Resolution Operator** or **MRO**. Which is basically a list of classes in order of precedence of method resolution. The first class in the list which contains the required method will be called. As seen from the example above, class `ParentA` is first in the list which contains the said method so the `say_hello` of `ParentA` is called.

## MRO / C3 Linearization

The MRO list seen above is generated using a linearization algorithm called C3 linearlization algorithm. A linearlization algorithm is a algorithm which takes a non linearlization / non list like data structure and converts it a linear data structure / list like data structure. 

C3 was introduced in python from the version 2.3. Before C3 depth first, left to right traversal was used as a way to find the mro sequence. The problem with depth first, left to right was that the algorithm was not monotonic. A MRO is monotonic when the following is true: if C1 precedes C2 in the linearization of C, then C1 precedes C2 in the linearization of any subclass of C. Otherwise, the innocuous operation of deriving a new class could change the resolution order of methods, potentially introducing very subtle bugs. Thus C3 was introduced which provided monotonic mro for any complex class hierarchy.

Before understanding C3 there are few terminology that we will use,

1) **merge**: Merge is a function which take N number of list as an input and combines them into a single list removing the duplicate.

2) **Head**: Head is the first element of any list.

3) **Tail**: Tail is part of list which is left after removing the head.

4) **Good Head**: A Good head is a head which only appears on the start of the list but not in the between or end of the list.

Lets take an example.

In [8]:
class O(object): pass

class A(O): pass

class B(O): pass

class C(O): pass

class D(O): pass

class E(O): pass

class K1(A, B, C): pass

class K2(D, B, E): pass

class K3(D, A): pass

class Z(K1, K2, K3): pass


print("Mro for class Z: ")
print(Z.mro())

Mro for class Z: 
[<class '__main__.Z'>, <class '__main__.K1'>, <class '__main__.K2'>, <class '__main__.K3'>, <class '__main__.D'>, <class '__main__.A'>, <class '__main__.B'>, <class '__main__.C'>, <class '__main__.E'>, <class '__main__.O'>, <class 'object'>]


For any class we can computer the mro with following formula

$$
L[class] = class + merge(L[parents], \text{ list of parents})
$$

where $L$ is the linearlization of the class.

So for Class `O` it is simple.

$$
L[O] = O \tag{as there is no parent}
$$

For class `A`, we compute MRO as follows.

$$
\begin{align*}
L[A] &= A + merge(L[O], O) \tag{A as only one parent} \\
L[A] &= A + merge([O], O) \tag{from above} \\
L[A] &= [A, O] \tag{Single entry with duplicate in merge}
\end{align*}
$$

Similarly we can find,

$$
\begin{align*}
L[B] &= [B, O] \\
L[C] &= [C, O] \\
L[D] &= [D, O] \\
L[E] &= [E, O]
\end{align*}
$$

Now for class `K1`.

$$
\begin{align*}
L[K1] &= K1 + merge(L[A], L[B], L[C], A, B, C) \\
L[K1] &= K1 + merge([A, O], [B, O], [C, O], A, B, C) \tag{From previous result} \\ 
\end{align*}
$$

Now here we select the head in merge `A`. `A` is a good head, because `A` only appears in the beginning of all the list. So we remove A from the merge list and add it to the final MRO list thus getting.

$$
\begin{align*}
L[K1] &= [K1, A] + merge([O], [B, O], [C, O], B, C) \\ 
\end{align*}
$$

Now the next head is `O`. But `O` is not a good head, because it appears in the end of the list $[B, O]$ and $[C, O]$. So we select the next head which is `B`. `B` is a good head because it appears only at the starting of the list and nowhere else. Thus adding `B` to our MRO list and thus similarly we will get the following.

$$
\begin{align*}
L[K1] &= [K1, A, B] + merge([O], [O], [C, O], C) \\ 
L[K1] &= [K1, A, B, C] + merge([O], [O], [O]) \tag{C as good head}\\
L[K1] &= [K1, A, B, C, O] \tag{Single entry list} \\ 
\end{align*}
$$

Similarly for `K2` and `K3` we can find.

$$
\begin{align*}
L[K2] &= [K2, D, B, E, O] \\ 
L[K3] &= [K3, D, A, O]
\end{align*}
$$

Now for class `Z`. we do the following.

$$
\begin{align*}
L[Z] &= Z + merge(L[K1], L[K2], L[K3], K1, K2, K3) \\ 
L[Z] &= Z + merge([K1, A, B, C, O], [K2, D, B, E, O] , [K3, D, A, O], K1, K2, K3) \tag{From above}\\
\end{align*}
$$

Here `K1` is a good head as it only appears at the start of all the list and nowhere else. so we select `K1` as a good head and remove from merge list and add it to the MRO list.

$$
\begin{align*}
L[Z] &= [Z, K1 ] + merge([A, B, C, O], [K2, D, B, E, O] , [K3, D, A, O], K2, K3)\\
\end{align*}
$$

Now next head is `A` which is a bad head because `A` appears in the middle of the list $[K3, D, A, O]$ so we skip `A`. Similarly we skip `B`, `C` and `O` as well because the appear in the middle or the end of the list. Now we select `K2`, which is a good head because it appears only at the start of the list, so we remove `K2` and add it to the MRO list.

$$
\begin{align*}
L[Z] &= [Z, K1, K2] + merge([A, B, C, O], [D, B, E, O] , [K3, D, A, O], K3)\\
\end{align*}
$$

For similarly reason `K3` is good head.

$$
\begin{align*}
L[Z] &= [Z, K1, K2, K3] + merge([A, B, C, O], [D, B, E, O] , [D, A, O]) \\
\end{align*}
$$

Now we repeat until the merge list is empty. So the next head is `A` is which bad because it appears in middle of $[D, A, O]$. Also `B`, `C` and `O` are bad because the appear in the middle or the end of the list. But head `D` is good as it only appears at the start of all the lists, so we select `D` and add it to the MRO list and remove it from merge list. And following the same steps we get the following result.


$$
\begin{align*}
L[Z] &= [Z, K1, K2, K3, D] + merge([A, B, C, O], [B, E, O] , [A, O]) \tag{D as good head} \\
L[Z] &= [Z, K1, K2, K3, D, A] + merge([B, C, O], [B, E, O] , [O]) \tag{A as good head} \\
L[Z] &= [Z, K1, K2, K3, D, A, B] + merge([C, O], [E, O] , [O]) \tag{B as good head} \\
L[Z] &= [Z, K1, K2, K3, D, A, B, C] + merge([O], [E, O] , [O]) \tag{C as good head} \\
L[Z] &= [Z, K1, K2, K3, D, A, B, C, E] + merge([O], [O] , [O]) \tag{E as good head} \\
L[Z] &= [Z, K1, K2, K3, D, A, B, C, E, O]  \tag{Single entry in merge} \\
\end{align*}
$$

Thus we get the MRO for class `Z`.

$$
\begin{align*}
L[Z] &= [Z, K1, K2, K3, D, A, B, C, E, O]  \\
\end{align*}
$$
